### LOAD DATA INTO SQLITE

In [1]:
import sqlite3
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#Load Dataset 
df = pd.read_csv('github_repos_full.csv')

#Create SQL Connection
con = sqlite3.connect('github_repos_full.db')

#Store into table 
df.to_sql("github_data",con, if_exists = "replace", index= False )

3000

In [2]:
query = "select * from Respo Limit 3; "
pd.read_sql_query(query, con)

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions,score
0,28457823,MDEwOlJlcG9zaXRvcnkyODQ1NzgyMw==,freeCodeCamp,freeCodeCamp/freeCodeCamp,0,"{'login': 'freeCodeCamp', 'id': 9892522, 'node...",https://github.com/freeCodeCamp/freeCodeCamp,freeCodeCamp.org's open-source codebase and cu...,0,https://api.github.com/repos/freeCodeCamp/free...,...,0,0,"['careers', 'certification', 'community', 'cur...",public,41197,298,425796,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
1,132750724,MDEwOlJlcG9zaXRvcnkxMzI3NTA3MjQ=,build-your-own-x,codecrafters-io/build-your-own-x,0,"{'login': 'codecrafters-io', 'id': 58904235, '...",https://github.com/codecrafters-io/build-your-...,Master programming by recreating your favorite...,0,https://api.github.com/repos/codecrafters-io/b...,...,0,0,"['awesome-list', 'free', 'programming', 'tutor...",public,38494,408,410406,master,"{'admin': False, 'maintain': False, 'push': Fa...",1.0
2,21737465,MDEwOlJlcG9zaXRvcnkyMTczNzQ2NQ==,awesome,sindresorhus/awesome,0,"{'login': 'sindresorhus', 'id': 170270, 'node_...",https://github.com/sindresorhus/awesome,😎 Awesome lists about all kinds of interesting...,0,https://api.github.com/repos/sindresorhus/awesome,...,0,0,"['awesome', 'awesome-list', 'lists', 'resource...",public,31169,52,391821,main,"{'admin': False, 'maintain': False, 'push': Fa...",1.0


In [3]:
#Extract Useful Columns

def columns(con, table_name, columns):
    col_str=",".join(columns)
    query = f"select {col_str} from {table_name};"
    return pd.read_sql_query(query, con)

col_name = ['full_name', 'language', 'stargazers_count', 'forks_count', 'open_issues_count', 'watchers_count', 'created_at', 'updated_at',
            'pushed_at', 'license']

df_selected_col = columns(con, "Respo", col_name)
df_selected_col.head(3)


,full_name,language,stargazers_count,forks_count,open_issues_count,watchers_count,created_at,updated_at,pushed_at,license
0,freeCodeCamp/freeCodeCamp,TypeScript,425796,41197,298,425796,2014-12-24T17:49:19Z,2025-08-15T11:47:40Z,2025-08-15T09:34:45Z,"{'key': 'bsd-3-clause', 'name': 'BSD 3-Clause ..."
1,codecrafters-io/build-your-own-x,Markdown,410406,38494,408,410406,2018-05-09T12:03:18Z,2025-08-15T11:53:48Z,2025-07-29T21:02:41Z,None
2,sindresorhus/awesome,None,391821,31169,52,391821,2014-07-11T13:42:37Z,2025-08-15T11:57:08Z,2025-07-18T18:37:33Z,"{'key': 'cc0-1.0', 'name': 'Creative Commons Z..."


In [4]:
#Handling Missing Values

df_selected_col.isnull().sum()

full_name              0
language             277
stargazers_count       0
forks_count            0
open_issues_count      0
watchers_count         0
created_at             0
updated_at             0
pushed_at              0
license              414
dtype: int64

### DATA CLEANING USING PYTHON

In [5]:
import pandas as pd
import re

#1. Handling missing value
def fill_missing_unknown(df):
    # Replace null/NaN values
    df["language"] = df["language"].fillna("Unknown")
    df["license"] = df["license"].fillna("Unknown")
   
    return df
    

In [6]:
# 2. Convert date columns
def convert_to_date(df):
    df['created_at'] = pd.to_datetime(df_selected_col['created_at']).dt.date
    df['updated_at'] = pd.to_datetime(df_selected_col['updated_at']).dt.date
    df['pushed_at'] = pd.to_datetime(df_selected_col['pushed_at']).dt.date
    return df


In [7]:
# 3. Split full_name into owner and repo
def split_name(df):
    df[["owner_name", "repo_name"]] = df["full_name"].str.split("/", n=1, expand=True)
    return df


In [8]:
# 4. Extract license name
def licence_name(df):
    def extract_license(text):
        if text is None:
            return "Unknown"
        match = re.search(r"'name':\s*'([^']+)'", str(text))
        return match.group(1) if match else "Unknown"

    df["license_name"] = df["license"].apply(extract_license)
    return df
    

In [9]:
# 5. Categorize license
categories = {
    "Permissive": [
        "Apache License 2.0", "BSD 2-Clause \"Simplified\" License",
        "BSD 3-Clause \"New\" or \"Revised\" License", "ISC License",
        "MIT License", "MIT No Attribution", "Microsoft Public License",
        "Mozilla Public License 2.0", "Universal Permissive License v1.0",
        "zlib License"
    ],
    "Copyleft": [
        "GNU Affero General Public License v3.0",
        "GNU General Public License v2.0",
        "GNU General Public License v3.0",
        "GNU Lesser General Public License v2.1",
        "GNU Lesser General Public License v3.0"
    ],
    "Special-use": [
        "Creative Commons Attribution 4.0 International",
        "Creative Commons Attribution Share Alike 4.0 International",
        "Creative Commons Zero v1.0 Universal",
        "Do What The F*ck You Want To Public License",
        "LaTeX Project Public License v1.3c",
        "SIL Open Font License 1.1",
        "The Unlicense",
        "Vim License"
    ],
    "Other/Unknown": ["Other", "Unknown"]
}

def map_license_to_category(df):
    def get_category(license_name):
        for category, license_list in categories.items():
            if license_name in license_list:
                return category
        return "Uncategorized"
    df["license_name"] = df["license_name"].apply(get_category)
    return df


### PIPELINE

In [10]:
# --- PIPELINE ---
def clean_github_data(con, table_name):
    # Load raw table
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", con)

    # Apply cleaning steps in sequence
    df = fill_missing_unknown(df)
    df = convert_to_date(df)
    df = split_name(df)
    df = licence_name(df)
    df = map_license_to_category(df)




    # Final cleaned columns
    return df[[
        "owner_name", "repo_name", "language", "license_name",
        "stargazers_count", "forks_count", "open_issues_count",
        "watchers_count", "created_at", "updated_at", "pushed_at"
    ]]

df_cleaned = clean_github_data(con, 'github_data')
df_cleaned.head().T

,0,1,2,3,4
owner_name,freeCodeCamp,codecrafters-io,sindresorhus,EbookFoundation,public-apis
repo_name,freeCodeCamp,build-your-own-x,awesome,free-programming-books,public-apis
language,TypeScript,Markdown,Unknown,Python,Python
license_name,Permissive,Other/Unknown,Special-use,Special-use,Permissive
stargazers_count,426376,413794,394655,366175,362792
forks_count,41305,38805,31337,63964,38078
open_issues_count,311,419,55,39,558
watchers_count,426376,413794,394655,366175,362792
created_at,2014-12-24,2018-05-09,2014-07-11,2013-10-11,2016-03-20
updated_at,2025-08-15,2025-08-15,2025-08-15,2025-08-15,2025-08-15


In [11]:
df_cleaned.isnull().sum()

owner_name           0
repo_name            0
language             0
license_name         0
stargazers_count     0
forks_count          0
open_issues_count    0
watchers_count       0
created_at           0
updated_at           0
pushed_at            0
dtype: int64

In [12]:
# Save back to the same table (overwrite old data)
df_cleaned.to_sql("github_data", con, if_exists="replace", index=False)

print("Cleaned data saved successfully to 'github_data' table ✅")


Cleaned data saved successfully to 'github_data' table ✅


In [13]:
# Save as CSV file

df_cleaned.to_csv("cleaned_github_data", index = False)
print("Successfully saved 'cleaned_github_data.csv' file")

Successfully saved 'cleaned_github_data.csv' file
